# vectorisation

In [26]:
import numpy as np

Let us make some dummy data $X$. This is a matrix that contains $n$ observations, each observations having $k$ features, thus forming an $n \times k$ matrix.

In [27]:
np.random.seed(42)
n = 1000
k = 3
X = np.random.randint(10, size=(n, k))

print(X)

[[6 3 7]
 [4 6 9]
 [2 6 7]
 ...
 [2 0 6]
 [9 4 4]
 [0 8 6]]


In this example, the first observation consists of three features. The first feature has a value of 6, the second of 3, the third of 7. To make predicication, we will want to learn what weight we should assign to every feature. Not every feature will be as important in predicting an outcome. This will depend on what we want to predict.

However, at the beginning of each learning algorithm we will randomize the weights.

In [28]:
np.random.seed(42)
W = np.random.rand(k, 1)
W

array([[0.37454012],
       [0.95071431],
       [0.73199394]])

The goal will be to learn the proper weights for our type of prediction.

E.g. If the features are length, weight and body temperature, the first two features will be roughly equally important in predicting the size of someones clothes, while the third will be of no importance. 

The final weights for this case might look something like $[0.9, 0.8, 0.01]$

However, for predicting shoe size, the first feature will be more important than the last one. We might get something like $[0.9, 0.3, 0.01]$. 

For predicting infections, the last feature will be the most imporant, e.g. $[0.01, 0.1, 0.9]$.



If we want to make a prediction, often denoted as $\hat{y}$ (y-hat), a very general way to do so is to multiply every feature with a weight:

\begin{equation}
\hat{y} = w_1 x_1 + \dots + w_n x_n
\end{equation}

This approach is a basic element in many machine learning algorithms. We might add some extra tricks, like Baysian Statistics, SVM's or stacking Neural Network layers upon eachother.

A naive way to calculate this, would be while using a forloop. Making an empty matrix of zeros with the right shape is much faster then appending an outcome to a list.

In [29]:
def for_loop(X, W, result):
    # for every observation
    for i, observation in enumerate(X):
        yhat = 0
        # we want to multiply every feature with a weight
        for j, feature in enumerate(observation):
            # and make a summation of every weigth * feature
            yhat += W[j] * feature
        result[i, 0] = yhat
    return(result)

In [30]:
result = np.zeros((X.shape[0],W.shape[1]))

looptime = %timeit -o for_loop(X, W, result)

15 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
np.random.seed(42)
n = 10000
k = 10
X = np.random.randint(10, size=(n, k))
W = np.random.rand(k, 1)
result = np.zeros((X.shape[0],W.shape[1]))
looptime = %timeit -o for_loop(X, W, result)

457 ms ± 38.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Probably, you will have a result somewhere in the millisecond range. With $n=1000$ and $k=3$, my laptop returns about 10 ms. Scaling this to n = 10000 and k = 10 gives about 300 ms. Even though this might seem fast, let's compare this to matrix multiplication.

Note that, for matrix multiplication to work, we will need the dimensions of the two matrices to match. If our first matrix $X$ has dimensions $(n, k)$ and our second matrix $W$ has dimensions $(k, m)$, we are able to do $X \dot W$ because the dimension $k$ is equal.

So, trying to do a matrix multiplication with dimensions (10, 3) and (3, 1) will work, but (10,3) and (4,1) will fail because $3\neq 4$.

Also, trying to multiply (3,1) and (10, 3) will fail. This means the order is important. This is different from normal multiplication, where $2 \times 3$ and $3 \times 2$ both equal to 6.

Let's say our data has $n=2$ cases ($a$ and $b$), where we observe $k=2$ features for every case. We now need two weights $w_1$ and $w_2$, one for every feature. A matrix multiplication would do exactly what we did with the forloop:

$$
\begin{bmatrix}
a_{1} & a_2 \\
b_{1} & b_{2}
\end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2
\end{bmatrix}
=
\begin{bmatrix}
a_1*w_1 + a_2*w_2\\
b_1*w_1 + b_2*w_2
\end{bmatrix}
$$

If we implement the same formula, we see that everything gets much more compact.

In [33]:
def vectorisation(X, W):
    return np.dot(X, W)

In [34]:
np.random.seed(42)
n = 10000
k = 10
X = np.random.randint(10, size=(n, k))
W = np.random.rand(k, 1)
vectortime = %timeit -o vectorisation(X, W)

105 µs ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In addition to more compact code, we get a huge speed improvement.
$n=10000$ and $k=10$ with matrix multiplication gives a result around 60 microseconds. That is more then a factor 1000 faster!

In [35]:
np.mean(looptime.timings) / np.mean(vectortime.timings)

4336.634931970708

Let's double check we got the same outcome:

In [36]:
result = np.zeros((X.shape[0],W.shape[1]))
np.sum(for_loop(X, W, result) - vectorisation(X, W))

1.6253665080512292e-12

while there is a very small difference, this is due to rounding errors and the **summed** error is smaller dan $10^{-12}$. This is due to rouding errors.

This same principle works for a lot of numpy functions. Let's say we want to take the sinus of 100 values in a vector. Instead of going through a forloop like this:

In [37]:
X = np.random.rand(100)
%timeit for x in X: np.sin(x)

170 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


We can simply do this

In [38]:
%timeit np.sin(X)

1.89 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Which gives us the exact same results.

In addition to this, we don't have to worry too much about adding extra dimensions. Numpy simply scales things:

In [39]:
X = np.random.rand(100,100)
np.log(X)
np.exp(X)
X = np.random.rand(10, 10, 10)
np.sin(X)

array([[[0.8239886 , 0.71848961, 0.06993476, 0.05604759, 0.04581963,
         0.659005  , 0.49135754, 0.14744814, 0.80735118, 0.19946092],
        [0.6607936 , 0.84065827, 0.83632242, 0.58049584, 0.15250417,
         0.83756552, 0.6644321 , 0.73585227, 0.83124777, 0.2018063 ],
        [0.31665034, 0.24345891, 0.83654922, 0.60873393, 0.30560616,
         0.36193417, 0.43862713, 0.42468639, 0.1726946 , 0.75554553],
        [0.69784989, 0.79172379, 0.50166134, 0.6225348 , 0.68454508,
         0.5599242 , 0.6473771 , 0.509019  , 0.38890519, 0.8321013 ],
        [0.45959484, 0.74324548, 0.56301806, 0.2690009 , 0.11266205,
         0.79259332, 0.33093175, 0.29106101, 0.67959815, 0.60198278],
        [0.61365912, 0.13135068, 0.24556778, 0.47576609, 0.76354648,
         0.41268109, 0.14133345, 0.38970878, 0.28748325, 0.83853645],
        [0.3670788 , 0.57743551, 0.54361226, 0.16926574, 0.12976418,
         0.01766871, 0.13697929, 0.66438876, 0.59128406, 0.2802359 ],
        [0.21356107, 0.0314

A variation on matrix multiplication is element wise multiplication, also known as the Hadamard product. Compare the two examples below:

In [40]:
A = np.array([[1,2],
              [3,4]])

B = np.array([[1,2],
              [3,4]])
np.dot(A, B)

array([[ 7, 10],
       [15, 22]])

Here, we calculate $(1 \times 1) + (2 \times 3)$ for the first entry, which is 7.

However, what if we wanted to multiply every element with the same element in the other matrix (so, every element at position $(i,j)$ in $A$ is multiplied with every $(i,j)$ element in $B$).

In [41]:
np.multiply(A, B)

array([[ 1,  4],
       [ 9, 16]])

This equals to:

In [ ]:
A * B

# Broadcasting

Broadcasting referes to how numpy handles arrays with different shapes. In general, the smaller array is "broadcast" across the larger array to get compatible shapes. Multiplying 1-dimensional array of the same length performs a element-wise multiplication:

In [42]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([1.0, 2.0, 3.0])
a * b 

array([1., 4., 9.])

However, if the dimensions don't match, we get an error:

In [43]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([1.0, 2.0, 3.0, 4.0])
try:
     a * b 
except Exception as e:
    print('Error:', e)

Error: operands could not be broadcast together with shapes (3,) (4,) 


But this constraint is relaxed when the dimensions meet certain constraints:

In [44]:
a = np.array([1.0, 2.0, 3.0])
b = 2.0
a * b 

array([2., 4., 6.])

Which is equivalent to

In [45]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a * b 

array([2., 4., 6.])

But just shorter. In the first case, we can think of `b` being strethced to an array of the same shape as `a`.

In general, two dimensions are compatible when:
1. they are equal
2. one of them is 1

In addition to this, array's do not need to have the same number of dimensions. Eg an image with dimensions of `256x256` pixels can have three colors. This results in a `256x256x3` shape matrix. If we want to scale each color, we can use a 1-dimensional array with 3 values.

In [48]:
image = np.random.rand(256, 256, 3)
scale = np.array([2, 3, 6])
output = image * scale
output.shape

(256, 256, 3)

In [49]:
scale

array([2, 3, 6])

This "broadcasting" works along multiple dimensions

In [50]:
A = np.random.rand(15, 3, 5)
B = np.random.rand(15, 1, 5)
C = A * B
C.shape

(15, 3, 5)

Or more complex:

In [51]:
A = np.random.rand(8, 1, 6, 1)
B = np.random.rand(7, 1, 5)
C = A*B
C.shape

(8, 7, 6, 5)

Can you figure out what happened here?

# Exercise
You receive data from 1000 patients. For every patient, there are 8 features.

In [52]:
np.random.seed(42)
n = 1000
k = 8
X = np.random.rand(n, k)
X.shape

(1000, 8)

You want to build a simple linear model, and start by implementing the formula $f(x) = \sum_{i=1}^n w_i * x_i $

First, you initialize random weights. If you want to use matrix multiplication, what is the shape you need for the weights?

In [53]:
# shape van X: (1000,8)
#shape W: (8,2)
# shape np.dot(X, W)? (1000, 2)
W = np.random.rand(8,1)

If you multiply $X$ with $W$, what do you expect the shape of the outcome to be? In the outcome, what is the meaning of every row? And every column? Implement the forumula by using `np.dot`. Did you get the dimensions right?

In [54]:
y = np.dot(X, W)
y.shape

(1000, 1)

Now, assume that instead of predicting one outcome, you want to predict two different things. Every patient still has 8 features, but the outcome should be two numbers, instead of one.

How do you need to change the shape of $W$?

What is the expected shape if you apply matrix multiplication between $X$ and $W$? What would happen if you multiply $W$ and $X$?

Create $W$ and implement the new calculation. How does this differ from the calculation with just one outcome?

In [56]:
W = np.random.rand(8,4)
y = np.dot(X, W)
y.shape

(1000, 4)